# Rules of Ultimate RAG
We want to create a way to query the rules of Ultimate using **retrieval augmented generation** (RAG).

In [23]:
# from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.load import dumps, loads
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import WebBaseLoader
import bs4

from operator import itemgetter
from dotenv import load_dotenv
import os

# import secrets as environment variables
load_dotenv()
os.environ['LANGCHAIN_TRACING_V2'] = 'true' # whether to track using langsmith
os.environ['LANGCHAIN_ENDPOINT'] ="https://api.smith.langchain.com"
os.environ['LANGCHAIN_PROJECT'] = 'Trevor learning'

In [2]:
# Convert PDF files to text
import fitz # install using: pip install PyMuPDF

def extract_text_from_pdf(pdf_file_path: str):
    """Extracts the text from a pdf file
    Args:
        pdf_file_path (str): path to pdf file to use
    Returns:
        str: the text from the file
    """
    with fitz.open(pdf_file_path) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
    return text


## Load rules
Can use local files (pdf) or from a website

In [3]:
# # Download rules to /texts folder
# ! mkdir texts
# ! wget https://usaultimate.org/wp-content/uploads/2022/01/Official-Rules-of-Ultimate-2022-2023.pdf -P texts

In [33]:
# load rules from PDF

pdf_file_path = "texts/Official-Rules-of-Ultimate-2022-2023.pdf"
rules_text = extract_text_from_pdf(pdf_file_path)
print(f"Start of `rules_text`: \n{rules_text[0:200]}")

# Split text into chunks
chunk_size=2000
chunk_overlap=100
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
splits = text_splitter.create_documents((rules_text,))

# Store embeddings
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever(
    search_type="mmr"
)


Start of `rules_text`: 
 
2022-23 Official Rules of Ultimate 
 
Preface 
Ultimate is a sport that inspires players and fans alike because of its ability to develop and showcase the 
athleticism, skill, teamwork, and characte


In [34]:
template = """You are a rules expert for the sport of ultimate, sometimes called ultimate frisbee. 
You will be given the official rules as context and will answer questions based on the rules.
Answers should be no more than three sentences long. 
If you don't know the answer say that you don't.

Answer the following question based on this context:
{context}
Question: 
{question}
"""
prompt = ChatPromptTemplate.from_template(template)

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
simple_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [35]:
# Question
question = "Please explain how personal misconduct fouls work"
answer = simple_rag_chain.invoke(question)
print(answer)

Personal Misconduct Fouls (Yellow Cards) are assessed against a specific team member for particularly egregious conduct or a pattern of such behavior. A PMF is a formal warning for unacceptable behavior and puts the team member on notice that any further such actions will result in ejection from the game. If a team member receives a second PMF during a single game, they are ejected for the remainder of that game.


# Using Query Fusion to get better results

In [36]:
# Generate multiple rewordings of the question
n_rewordings = 5
template = f"""You are an AI language model assistant. Your task is to generate {n_rewordings} 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. 
All questions will relate to the sport of ultimate, sometimes called ultimate frisbee.
Original question: {{question}}"""
prompt_alternatives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_alternatives 
    | ChatOpenAI(temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)
# Try it out
generate_queries.invoke({"question":question})

['1. How are personal misconduct fouls handled in ultimate frisbee?',
 '2. Can you elaborate on the rules regarding personal misconduct fouls in ultimate?',
 '3. What happens when a player commits a personal misconduct foul in ultimate frisbee?',
 '4. Could you provide a detailed explanation of the consequences of personal misconduct fouls in ultimate?',
 '5. What are the protocols for addressing personal misconduct fouls in the sport of ultimate frisbee?']

In [37]:
def reciprocal_rank_fusion(results: list[list], top_n=5, k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula, 
        returning the top_n highest ranked results"""

    fused_scores = {}
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    # return top_n highest ranked results
    if len(reranked_results)> top_n:
        reranked_results=reranked_results[0:top_n]
    return reranked_results


In [38]:
# get documents for each question then rank them
retrieval_chain_rag_fusion = (
    generate_queries 
    | retriever.map() 
    | reciprocal_rank_fusion
)
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

5

In [39]:
# RAG
template = """You are a rules expert for the sport of ultimate, sometimes called ultimate frisbee. 
You will be given the official rules as context and will answer questions based on the rules.
Answers should be one to four sentences long, but keep it as short as possible. 
If you don't know the answer say that you don't.

Answer the following question based on this context:
{context}
Question: 
{question}
"""
prompt = ChatPromptTemplate.from_template(template)
final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)
final_rag_chain.invoke({"question":question})

'Personal misconduct fouls result in ejection from the game. The ejected player must leave the area and refrain from interacting with team members, spectators, or officials. Failure to comply may result in a forfeit for the team.'

In [40]:
docs

[(Document(page_content='This appendix describes the system for handling misconduct within a game that is using observers. \nAdditional mechanisms for handling misconduct at the event level or beyond are described in the USA \nUltimate Conduct Policy. The “Acting Disciplinary Authority” (ADA) as referred to in this Appendix means \nany tournament director, Tournament Rules Group, certified observer, Sectional or Regional Coordinator, \nState Youth Coordinator, Regional Youth Director, National Director, member of the USA Ultimate \nAdministration performing their duties in some specific capacity, officer or agent of USA Ultimate, or some \nother USA Ultimate-appointed individual or group charged with decision-making regarding conduct \npertaining to a specific USA Ultimate event or program in progress. \nB1. Components of the Misconduct System  \nA. Technical Foul  \n1. A Technical Foul can be assessed against a team for minor conduct violations that do not \naffect the competitiveness